Trong file này, em thực hiện y hệt cài đặt trong file HMM.ipynb. Điều khác biệt duy nhất là em chỉ sử dụng các label: ["len", "xuong", "phai", "trai", "nhay"] (ngoại trừ "sil", "A", "B" và "ban").

Lý do cho việc làm như vậy vì em muốn thử thu nhỏ lại để thấy tình huống mà ma trận mixture covariance không bị suy biến. Ngoài ra nhóm em có ý tưởng làm bài tập cuối kì chỉ sử dụng các nhãn ["phai", "trai", "nhay"] nên em muốn đánh giá sơ lược khả năng mô hình có khả quan không

# Khai báo thư viện và các đường dẫn cần thiết

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import fnmatch
import pandas as pd
import os.path
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
!pip install hmmlearn
import hmmlearn.hmm as hmm
from sklearn.utils import shuffle

     |████████████████████████████████| 129 kB 10.1 MB/s 


In [2]:
from google.colab import drive
drive.mount("/content/drive")
DATA_ROOT_DIR="/content/drive/MyDrive/Speech/Data_14/"
!ls $DATA_ROOT_DIR
DATA_MFCC_DIR=DATA_ROOT_DIR+"MFCC/"
!ls $DATA_MFCC_DIR

Mounted at /content/drive
A  ban	MFCC	     model.pkl		nhay  trai
B  len	model_1.pkl  model_without.pkl	phai  xuong
A_mfcc.npy    B_mfcc.npy    nhay_mfcc.npy  trai_mfcc.npy
ban_mfcc.npy  len_mfcc.npy  phai_mfcc.npy  xuong_mfcc.npy


# Xử lý dữ liệu
Trong mục này, em chuẩn bị dữ liệu để huấn luyện và đánh giá mô hình

- Load dữ liệu từ file đặc trưng MFCC đã tính toán trước đó
- Chia tập dữ liệu train và test với tỷ lệ 70:30

In [3]:
class_names = ["len", "xuong", "phai", "trai", "nhay"]
features = []
labels = []

#Với từng label
for cname in class_names:
  
  #Load file MFCC
  mfcc_data = np.load(DATA_MFCC_DIR+cname+"_mfcc.npy",allow_pickle=True)
  len_label = mfcc_data.shape[0]

  #Thêm các đặc trưng vào list chung
  for i in range(len_label):
    labels.append(cname)
    features.append(mfcc_data[i])
print(len(features))
print(len(labels))

2481
2481


In [65]:
#Thực hiện shuffle dữ liệu
c = list(zip(features, labels))
np.random.shuffle(c)
features,labels = zip(*c)

#Chia dữ liệu theo tỷ lệ 70:30
X_train , X_test , y_train , y_test  = train_test_split(features , 
                                                        labels, test_size = 0.3,
                                                        random_state=0)
print(len(X_train))
print(len(y_train))

1736
1736


In [4]:
#Lấy chỉ số tương ứng với label
gmmhmmdict = {}
index = 0
for cname in class_names:
  gmmhmmdict[cname] = index
  index = index +1
gmmhmmdict

{'len': 0, 'nhay': 4, 'phai': 2, 'trai': 3, 'xuong': 1}

# Chuẩn bị mô hình
Trong phần này, em thực hiện:
- Thiết lập các tham số đầu vào để huấn luyện mô hình
- Định nghĩa mô hình nhận các tham số đó

In [5]:
#Thiết lập các tham số cần thiết để train GMMHMM
m_num_of_HMMStates = 3  # số trạng thái
m_num_of_mixtures = 2  # số mixtures của mỗi hidden state
m_level = 2 #step size
m_covarianceType = 'diag'  # dạng ma trận covariance 
m_n_iter = 15  # số vòng lặp 

#khởi tạo giá trị cho startprobPrior và transmatPrior
def initLevel(inumstates, m_level):
    startprobPrior = np.zeros(inumstates)
    startprobPrior[0: m_level - 1] = 1/float((m_level - 1))
    transmatPrior = getTransmatPrior(inumstates, m_level)
    return startprobPrior, transmatPrior

#khởi tạo giá trị cho transmatPrior dựa vào step size
def getTransmatPrior(inumstates, m_level):
    transmatPrior = (1 / float(m_level)) * np.eye(inumstates)

    for i in range(inumstates - (m_level - 1)):
        for j in range(m_level - 1):
            transmatPrior[i, i + j + 1] = 1. / m_level

    for i in range(inumstates - m_level + 1, inumstates):
        for j in range(inumstates - i):
            transmatPrior[i, i + j] = 1. / (inumstates - i)

    return transmatPrior

startprobPrior ,transmatPrior = initLevel(m_num_of_HMMStates,m_level)

In [6]:
class SpeechModel:
  def __init__(self,Class,label):
    self.traindata = np.zeros((0,39))
    self.Class = Class
    self.label = label
    self.model  = hmm.GMMHMM(n_components = m_num_of_HMMStates, n_mix = m_num_of_mixtures,
                        transmat_prior = transmatPrior, startprob_prior = startprobPrior, 
                                    covariance_type = m_covarianceType, n_iter = m_n_iter, random_state=321)

# Huấn luyện mô hình
Với mỗi label em thực hiện huấn luyện một mô hình hmm tương ứng

In [81]:
speechmodels = [None] * len(class_names)

for key in gmmhmmdict:
  speechmodels[gmmhmmdict[key]] = SpeechModel(gmmhmmdict[key],key)

for i in range(0,len(X_train)):
  for j in range(0,len(speechmodels)):
    if int(speechmodels[j].Class) == int(gmmhmmdict[y_train[i]]):
      speechmodels[j].traindata = np.concatenate((speechmodels[j].traindata , X_train[i]))

In [82]:
for speechmodel in speechmodels:
    speechmodel.model.fit(speechmodel.traindata)

# Đánh giá mô hình bằng tập test
Trong phần này em thực hiện đánh giá mô hình sử dụng tập test đã chia bên trên. Nhãn dự đoán là nhãn ứng với mô hình mang lại score lớn nhất

In [83]:
y_pred = []

for i in range(0,len(X_test)):
    scores = []
    for speechmodel in speechmodels:
         scores.append(speechmodel.model.score(X_test[i]))
    id  = scores.index(max(scores))
    y_pred.append(speechmodels[id].label)
    # print(str(np.round(scores, 3)) + " " + str(max(np.round(scores, 3))) +" "+":"+ speechmodels[id].label)

In [84]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

         len       0.99      0.92      0.96       159
       xuong       0.64      0.99      0.78       181
        phai       0.97      0.60      0.74       128
        trai       0.96      0.66      0.78       140
        nhay       0.97      0.99      0.98       137

    accuracy                           0.85       745
   macro avg       0.91      0.83      0.85       745
weighted avg       0.89      0.85      0.85       745



In [85]:
from sklearn.metrics import accuracy_score
print('accuracy = ',accuracy_score(y_test, y_pred))

accuracy =  0.8483221476510067


In [86]:
trained_model = {}
for key in gmmhmmdict:
  trained_model[key] = speechmodels[gmmhmmdict[key]]
pickle.dump(trained_model, open(DATA_ROOT_DIR+'model_without.pkl', 'wb'))

# Demo nhận dạng với dữ liệu mới thu

In [11]:
trained_models = pickle.load(open(DATA_ROOT_DIR+'model_without.pkl', 'rb'))

In [12]:
def extract_mfcc_features(file_path,n_mfcc):
    #load file âm thanh
    sound, sr = librosa.load(file_path)

    #trích xuất đặc trưng
    mfcc = librosa.feature.mfcc(y=sound, sr=sr, n_mfcc=n_mfcc,hop_length=512)
    delta_mfcc = librosa.feature.delta(mfcc,width=9)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)

    #chuẩn hoá
    mfcc_features = np.concatenate((mfcc, delta_mfcc, delta2_mfcc)).T
    min_features = np.min(mfcc_features, axis=0)
    max_features = np.max(mfcc_features, axis=0)
    mfcc_features_nom = (mfcc_features - min_features) / (max_features - min_features)
    
    return mfcc_features_nom

In [13]:
n_mfcc=13
audio_file = "/content/drive/MyDrive/Speech/Data/nhay.wav"
ipd.Audio(audio_file)
# load audio files with librosa
signal, sr = librosa.load(audio_file)
mfccs_features = extract_mfcc_features(audio_file,n_mfcc)
mfccs_features.shape
# mfccs_features

(25, 39)

In [14]:
scores = []
for speechmodel in trained_models:
  scores.append(trained_models[speechmodel].model.score(mfccs_features))
id = scores.index(max(scores))

def get_key(val):
  for key, value in gmmhmmdict.items():
    if val == value:
      return key

print(get_key(id))

nhay
